<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Обзор-данных" data-toc-modified-id="Обзор-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Обзор данных</a></span><ul class="toc-item"><li><span><a href="#Объявление-функций" data-toc-modified-id="Объявление-функций-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Объявление функций</a></span></li><li><span><a href="#Распределение-типов-переменных" data-toc-modified-id="Распределение-типов-переменных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Распределение типов переменных</a></span></li></ul></li><li><span><a href="#Очистка-датасета" data-toc-modified-id="Очистка-датасета-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Очистка датасета</a></span><ul class="toc-item"><li><span><a href="#Отбираем-кат-признаки" data-toc-modified-id="Отбираем-кат-признаки-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Отбираем кат признаки</a></span></li><li><span><a href="#Отбираем-числовые-признаки" data-toc-modified-id="Отбираем-числовые-признаки-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Отбираем числовые признаки</a></span></li><li><span><a href="#Получившиеся-фичи" data-toc-modified-id="Получившиеся-фичи-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Получившиеся фичи</a></span></li></ul></li><li><span><a href="#get_features_importances" data-toc-modified-id="get_features_importances-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>get_features_importances</a></span><ul class="toc-item"><li><span><a href="#Downsample" data-toc-modified-id="Downsample-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Downsample</a></span></li><li><span><a href="#class_weight-=-'balanced'" data-toc-modified-id="class_weight-=-'balanced'-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>class_weight = 'balanced'</a></span></li><li><span><a href="#Отбираем-признаки" data-toc-modified-id="Отбираем-признаки-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Отбираем признаки</a></span></li></ul></li></ul></div>

# Отбор значимых непрерывных признаков

## Обзор данных

In [1]:
# импорт основных библиотек
import pandas as pd
import numpy as np

# импорт библиотек текущего модуля
from lightgbm import LGBMClassifier
from sklearn.model_selection import (
    train_test_split,
    GridSearchCV,
    cross_val_score,
)
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix
from sklearn.utils import resample
from scipy.stats import pointbiserialr
import phik

# настройки
pd.set_option("display.max_columns", 500)

# пути
dataset_train_path = "../../data/dataset_train.parquet"
dataset_test_path = "../../data/features_oot.parquet"
features_types_path = "../../data/features_types.json"
sample_submission_path = "../../data/sample_submission.csv"

# константы
RANDOM_STATE = 42

### Объявление функций

In [2]:
def read_data(path: str, left: int, right: int):
    """
    Функция для считывания определенного количества столбцов из parquet файла.
    ---
    Принимает на вход str путь к файлу, int левую и int правую границу.
    Столбец с таргетом добавляется в конец автоматически!
    Функция возвращает pd.DataFrame.
    """
    # следующие т столбцов, которые будем считывать + target
    columns_to_read = (
        features_types["feature"][left:right].to_list()
        + ["channel_name"]
        + ["target"]
    )

    # считываем столбцы в файл
    data = pd.read_parquet(path, engine="pyarrow", columns=columns_to_read)

    return data


def variance_filter(data: pd.DataFrame, threshold: float):
    """
    Функция для проверки датасета на наличие константных (или около того) значений.
    ---
    Принимает на вход pd.DataFrame и float с значением порога для дисперсии.
    Функция возвращает отфильтрованный pd.DataFrame.
    """
    # порог для дисперсии
    threshold = threshold

    # считаем дисперсию для каждого признака
    variance = data.iloc[:, :-1].var()

    # отбираем те фичи, у которых var > threshold
    selected_features = variance[variance > threshold].index.tolist()

    # собираем в датасет
    filtered_data = data[selected_features + ["target"]]

    return filtered_data


def check_corr_biserial(data: pd.DataFrame):
    """
    Функция для расчета коэф. корреляции бисериального всех признаков с целевой переменной.
    ---
    Принимает на вход pd.DataFrame.
    Возвращает pd.Series с значением коэффициента корреляции бисериального для каждого признака.
    """
    # все признаки, кроме последнего столбца с целевой переменной
    features = data.iloc[:, :-1]

    # пустой словарь для хранения результатов
    correlations = {}

    # проходимся по каждому признаку и вычисляем коэффициент корреляции
    for feature_name in features.columns:
        feature = data[feature_name]
        target = data["target"]
        correlation, p_value = pointbiserialr(feature, target)
        correlations[feature_name] = correlation

    # создаем DataFrame из словаря корреляций
    correlation = pd.DataFrame.from_dict(
        correlations, orient="index", columns=["point_biserial_correlation"]
    )

    # сортируем
    correlation = correlation.sort_values(
        by="point_biserial_correlation", ascending=False
    )

    return correlation


def check_corr_pearson(data: pd.DataFrame):
    """
    Функция для расчета корреляции Пирсона всех признаков с целевой переменной.
    ---
    Принимает на вход pd.DataFrame.
    Возвращает pd.Series с значением коэффициента корреляции для каждого признака.
    """
    # чекаем корреляцию с таргетом
    correlation = data.iloc[:, :-1].corrwith(data["target"])

    # сортируем
    correlation = correlation.sort_values(ascending=False)

    return correlation


def reshape_corr_matrix(data: pd.DataFrame):
    """
    Функция для преобразования вида корреляционной матрицы
    ---
    Принимает на вход pd.DataFrame (корр. матрицу)
    Возвращает pd.DataFrame со столбцами: ['feature_1', 'feature_2', 'corr']
    """
    # создаем пустой df
    reshaped_data = pd.DataFrame()

    # в цикле преобразовываем данные
    for feature in data.columns:
        temp_data = (
            data[feature]
            .reset_index()
            .rename(columns={"index": "feature_1", feature: "corr"})
        )

        temp_data["feature_2"] = feature

        reshaped_data = pd.concat([reshaped_data, temp_data])

    # меняем порядок столбцов
    reshaped_data = reshaped_data[["feature_1", "feature_2", "corr"]]

    # выводим результат
    return reshaped_data

### Распределение типов переменных

In [3]:
# читаем файлы
features_types = pd.read_json(features_types_path, orient="index")
sample_submission = pd.read_csv(sample_submission_path)

In [4]:
sample_submission


,id,target
0,0,0.343518
1,1,0.591216
2,2,0.913150
3,3,0.560035
4,4,0.352795
...,...,...
60656,60656,0.765319
60657,60657,0.533016
60658,60658,0.784497
60659,60659,0.804431


In [5]:
# приводим features_types к читаемому виду
features_types = features_types.reset_index().rename(
    columns={"index": "feature", 0: "type"}
)

# выводим на экран
features_types

,feature,type
0,markers_0_1_cnt,numeric
1,markers_1_1_cnt,numeric
2,markers_2_1_cnt,numeric
3,markers_3_1_cnt,numeric
4,markers_4_1_cnt,numeric
...,...,...
2771,markers_941_1_cnt,numeric
2772,markers_942_1_cnt,numeric
2773,markers_943_1_cnt,numeric
2774,markers_944_1_cnt,numeric


In [7]:
# распределение типов данных
features_types.type.value_counts()

type
numeric            2607
categorical_int     138
categorical_str      31
Name: count, dtype: int64

In [8]:
# смотрим на все фичи в паркете
print(features_types["feature"].to_list())

['markers_0_1_cnt', 'markers_1_1_cnt', 'markers_2_1_cnt', 'markers_3_1_cnt', 'markers_4_1_cnt', 'markers_5_1_cnt', 'markers_6_1_cnt', 'markers_7_1_cnt', 'markers_8_1_cnt', 'markers_9_1_cnt', 'markers_10_1_cnt', 'markers_11_1_cnt', 'markers_12_1_cnt', 'markers_13_1_cnt', 'markers_14_1_cnt', 'markers_15_1_cnt', 'markers_16_1_cnt', 'markers_17_1_cnt', 'markers_18_1_cnt', 'markers_19_1_cnt', 'markers_20_1_cnt', 'markers_21_1_cnt', 'markers_22_1_cnt', 'markers_23_1_cnt', 'markers_24_1_cnt', 'markers_25_1_cnt', 'markers_26_1_cnt', 'markers_27_1_cnt', 'markers_28_1_cnt', 'markers_29_1_cnt', 'markers_30_1_cnt', 'markers_31_1_cnt', 'markers_32_1_cnt', 'markers_33_1_cnt', 'markers_34_1_cnt', 'markers_35_1_cnt', 'markers_36_1_cnt', 'markers_37_1_cnt', 'markers_38_1_cnt', 'markers_39_1_cnt', 'markers_40_1_cnt', 'markers_41_1_cnt', 'markers_42_1_cnt', 'markers_43_1_cnt', 'markers_44_1_cnt', 'markers_45_1_cnt', 'markers_46_1_cnt', 'markers_47_1_cnt', 'markers_48_1_cnt', 'markers_49_1_cnt', 'markers_

## Очистка датасета

In [28]:
# считаем файл
dataset_train = read_data(dataset_train_path, 0, 930)
# dataset_test = pd.read_parquet(dataset_test_path, engine='pyarrow')


**НИЖЕ ОПЦИОНАЛЬНО: 'channel_name' == 3** 

---

In [29]:
# приведем к человеческому типу
dataset_train["channel_name"] = dataset_train["channel_name"].astype("int")

In [30]:
dataset_train["channel_name"].value_counts()

channel_name
5    221058
3    209022
4    135852
1    131566
2      4588
Name: count, dtype: int64

In [31]:
# фильтруем по channel_name == 3
dataset_train = dataset_train[dataset_train["channel_name"] == 3]
dataset_train

,markers_0_1_cnt,markers_1_1_cnt,markers_2_1_cnt,markers_3_1_cnt,markers_4_1_cnt,markers_5_1_cnt,markers_6_1_cnt,markers_7_1_cnt,markers_8_1_cnt,markers_9_1_cnt,markers_10_1_cnt,markers_11_1_cnt,markers_12_1_cnt,markers_13_1_cnt,markers_14_1_cnt,markers_15_1_cnt,markers_16_1_cnt,markers_17_1_cnt,markers_18_1_cnt,markers_19_1_cnt,markers_20_1_cnt,markers_21_1_cnt,markers_22_1_cnt,markers_23_1_cnt,markers_24_1_cnt,markers_25_1_cnt,markers_26_1_cnt,markers_27_1_cnt,markers_28_1_cnt,markers_29_1_cnt,markers_30_1_cnt,markers_31_1_cnt,markers_32_1_cnt,markers_33_1_cnt,markers_34_1_cnt,markers_35_1_cnt,markers_36_1_cnt,markers_37_1_cnt,markers_38_1_cnt,markers_39_1_cnt,markers_40_1_cnt,markers_41_1_cnt,markers_42_1_cnt,markers_43_1_cnt,markers_44_1_cnt,markers_45_1_cnt,markers_46_1_cnt,markers_47_1_cnt,markers_48_1_cnt,markers_49_1_cnt,markers_50_1_cnt,markers_51_1_cnt,markers_52_1_cnt,markers_53_1_cnt,markers_54_1_cnt,markers_55_1_cnt,markers_56_1_cnt,markers_57_1_cnt,markers_58_1_cnt,markers_59_1_cnt,markers_60_1_cnt,markers_61_1_cnt,markers_62_1_cnt,markers_63_1_cnt,markers_64_1_cnt,markers_65_1_cnt,markers_66_1_cnt,markers_67_1_cnt,markers_68_1_cnt,markers_69_1_cnt,markers_70_1_cnt,markers_71_1_cnt,markers_72_1_cnt,markers_73_1_cnt,markers_74_1_cnt,markers_75_1_cnt,markers_76_1_cnt,markers_77_1_cnt,markers_78_1_cnt,markers_79_1_cnt,markers_80_1_cnt,markers_81_1_cnt,markers_82_1_cnt,markers_83_1_cnt,markers_84_1_cnt,markers_85_1_cnt,markers_86_1_cnt,markers_87_1_cnt,markers_88_1_cnt,markers_89_1_cnt,markers_90_1_cnt,markers_91_1_cnt,markers_92_1_cnt,markers_93_1_cnt,markers_94_1_cnt,markers_95_1_cnt,markers_96_1_cnt,markers_97_1_cnt,markers_98_1_cnt,markers_99_1_cnt,markers_100_1_cnt,markers_101_1_cnt,markers_102_1_cnt,markers_103_1_cnt,markers_104_1_cnt,markers_105_1_cnt,markers_106_1_cnt,markers_107_1_cnt,markers_108_1_cnt,markers_109_1_cnt,markers_110_1_cnt,markers_111_1_cnt,markers_112_1_cnt,markers_113_1_cnt,markers_114_1_cnt,markers_115_1_cnt,markers_116_1_cnt,markers_117_1_cnt,markers_118_1_cnt,markers_119_1_cnt,markers_120_1_cnt,markers_121_1_cnt,markers_122_1_cnt,markers_123_1_cnt,markers_124_1_cnt,markers_125_1_cnt,markers_126_1_cnt,markers_128_1_cnt,markers_129_1_cnt,markers_131_1_cnt,markers_132_1_cnt,markers_133_1_cnt,markers_135_1_cnt,markers_136_1_cnt,markers_137_1_cnt,markers_138_1_cnt,markers_139_1_cnt,markers_140_1_cnt,markers_141_1_cnt,markers_142_1_cnt,markers_143_1_cnt,markers_145_1_cnt,markers_146_1_cnt,markers_147_1_cnt,markers_148_1_cnt,markers_149_1_cnt,markers_150_1_cnt,markers_151_1_cnt,markers_154_1_cnt,markers_155_1_cnt,markers_156_1_cnt,markers_157_1_cnt,markers_158_1_cnt,markers_159_1_cnt,markers_160_1_cnt,markers_161_1_cnt,markers_162_1_cnt,markers_163_1_cnt,markers_164_1_cnt,markers_165_1_cnt,markers_166_1_cnt,markers_167_1_cnt,markers_168_1_cnt,markers_169_1_cnt,markers_170_1_cnt,markers_171_1_cnt,markers_172_1_cnt,markers_173_1_cnt,markers_174_1_cnt,markers_175_1_cnt,markers_176_1_cnt,markers_177_1_cnt,markers_178_1_cnt,markers_179_1_cnt,markers_180_1_cnt,markers_181_1_cnt,markers_182_1_cnt,markers_183_1_cnt,markers_184_1_cnt,markers_185_1_cnt,markers_186_1_cnt,markers_187_1_cnt,markers_188_1_cnt,markers_189_1_cnt,markers_190_1_cnt,markers_191_1_cnt,markers_192_1_cnt,markers_193_1_cnt,markers_194_1_cnt,markers_195_1_cnt,markers_196_1_cnt,markers_198_1_cnt,markers_199_1_cnt,markers_201_1_cnt,markers_202_1_cnt,markers_203_1_cnt,markers_204_1_cnt,markers_205_1_cnt,markers_206_1_cnt,markers_207_1_cnt,markers_208_1_cnt,markers_209_1_cnt,markers_210_1_cnt,markers_211_1_cnt,markers_212_1_cnt,markers_213_1_cnt,markers_215_1_cnt,markers_216_1_cnt,markers_217_1_cnt,markers_218_1_cnt,markers_219_1_cnt,markers_220_1_cnt,markers_221_1_cnt,markers_222_1_cnt,markers_223_1_cnt,markers_224_1_cnt,markers_225_1_cnt,markers_226_1_cnt,markers_227_1_cnt,markers_228_1_cnt,markers_229_1_cnt,markers_230_1_cnt,markers_231_1_cnt,markers_232_1_cnt,markers_233_1_cnt,markers_234_1_cnt,markers_235_1_cnt,markers_236_1_cnt,marke

---

### Отбираем кат признаки

In [10]:
# получим список кат фичей
cat_features = []

for feature in dataset_train.columns[:-2]:
    if feature.split("_")[-1] in ["flg", "ctg"]:
        cat_features.append(feature)

cat_features

['materials_details_0_1_ctg',
 'communication_availability_0_1_flg',
 'communication_availability_1_1_ctg',
 'communication_availability_2_1_flg',
 'materials_details_1_1_ctg',
 'user_lifetime_0_1_ctg',
 'user_lifetime_1_1_flg',
 'communication_availability_3_1_flg',
 'materials_details_5_1_flg',
 'materials_details_7_1_flg',
 'materials_details_8_1_flg']

In [14]:
dataset_train[cat_features].info()


<class 'pandas.core.frame.DataFrame'>
Index: 209022 entries, 3 to 702075
Data columns (total 11 columns):
 #   Column                              Non-Null Count   Dtype
---  ------                              --------------   -----
 0   materials_details_0_1_ctg           209022 non-null  int64
 1   communication_availability_0_1_flg  209022 non-null  int32
 2   communication_availability_1_1_ctg  209022 non-null  int32
 3   communication_availability_2_1_flg  209022 non-null  int32
 4   materials_details_1_1_ctg           209022 non-null  int64
 5   user_lifetime_0_1_ctg               209022 non-null  int64
 6   user_lifetime_1_1_flg               209022 non-null  int32
 7   communication_availability_3_1_flg  209022 non-null  int32
 8   materials_details_5_1_flg           209022 non-null  int8 
 9   materials_details_7_1_flg           209022 non-null  int8 
 10  materials_details_8_1_flg           209022 non-null  int8 
dtypes: int32(5), int64(3), int8(3)
memory usage: 11.0 MB


In [15]:
# считаем корреляцию кт признаков между собой
cat_corr = dataset_train[cat_features].corr(method="spearman")

# удаляем категориальные признаки, которые сильно коррелируют друг с другом
cat_corr_reshaped = reshape_corr_matrix(cat_corr)

features_to_delete = cat_corr_reshaped.query(
    "corr != 1 and corr > 0.5"
).sort_values(by="corr", ascending=False)

features_to_delete

,feature_1,feature_2,corr
9,materials_details_7_1_flg,materials_details_8_1_flg,0.998024
10,materials_details_8_1_flg,materials_details_7_1_flg,0.998024
8,materials_details_5_1_flg,materials_details_7_1_flg,0.997431
9,materials_details_7_1_flg,materials_details_5_1_flg,0.997431
8,materials_details_5_1_flg,materials_details_8_1_flg,0.997426
10,materials_details_8_1_flg,materials_details_5_1_flg,0.997426
6,user_lifetime_1_1_flg,user_lifetime_0_1_ctg,0.975401
5,user_lifetime_0_1_ctg,user_lifetime_1_1_flg,0.975401
1,communication_availability_0_1_flg,communication_availability_3_1_flg,0.859481
7,communication_availability_3_1_flg,communication_availability_0_1_flg,0.859481


In [11]:
# отбираем кат признаки
cat_features = [
    "materials_details_8_1_flg",
    "user_lifetime_1_1_flg",
    "communication_availability_3_1_flg",
    "materials_details_1_1_ctg",
    "communication_availability_3_1_flg",
]

dataset_train[cat_features].corrwith(
    dataset_train.target, method="spearman"
).sort_values(ascending=False)

materials_details_8_1_flg             0.032019
communication_availability_3_1_flg    0.004628
communication_availability_3_1_flg    0.004628
user_lifetime_1_1_flg                 0.000252
materials_details_1_1_ctg            -0.017474
dtype: float64

### Отбираем числовые признаки

In [12]:
# теперь отберем числовые признаки
num_features = []

for feature in dataset_train.columns[:-2]:
    if feature.split("_")[-1] in ["num", "sum", "avg", "cnt"]:
        num_features.append(feature)

len(num_features)

907

Итого имеем отдельно категориальные и чистовые признаки!

Теперь очистим числовые с помощью дисперсии и корреляции

In [13]:
dataset_train[num_features].shape


(209022, 907)

In [14]:
# убираем константные столбцы
dataset_num = variance_filter(dataset_train[num_features + ["target"]], 0.5)
dataset_num.shape

(209022, 831)

Убрали ~ 70 признаков

In [15]:
# перезаписали
num_features = dataset_num.columns[:-1]


Теперь проведем корр анализ и также уберем лишние признаки

In [16]:
# установим трешхолд для корреляции
corr_threshold = 0.75


In [17]:
# считаем..
data_corr = dataset_train[num_features].corr()

# приводим в должный вид
reshaped_corr_matrix = reshape_corr_matrix(data_corr)


In [18]:
features_to_delete = (
    reshaped_corr_matrix.query("corr != 1.0 and corr > 0.70")
    .sort_values(by="corr", ascending=False)
    .iloc[::2, 1]
    .to_list()
)

len(features_to_delete)

333

In [19]:
dataset_num = dataset_num.drop(features_to_delete, axis=1)
dataset_num.shape


(209022, 729)

In [32]:
# опять перезаписали
num_features = list(dataset_num.columns[:-1])
len(num_features)


728

### Получившиеся фичи

In [33]:
my_test_features = num_features + cat_features + ["target"]
len(my_test_features)

734

In [34]:
dataset_train = dataset_train[my_test_features]
dataset_train.head()


,markers_0_1_cnt,markers_2_1_cnt,markers_3_1_cnt,markers_4_1_cnt,markers_5_1_cnt,markers_6_1_cnt,markers_7_1_cnt,markers_8_1_cnt,markers_9_1_cnt,markers_10_1_cnt,markers_11_1_cnt,markers_12_1_cnt,markers_13_1_cnt,markers_14_1_cnt,markers_15_1_cnt,markers_16_1_cnt,markers_17_1_cnt,markers_18_1_cnt,markers_19_1_cnt,markers_20_1_cnt,markers_21_1_cnt,markers_23_1_cnt,markers_24_1_cnt,markers_25_1_cnt,markers_26_1_cnt,markers_27_1_cnt,markers_28_1_cnt,markers_29_1_cnt,markers_31_1_cnt,markers_32_1_cnt,markers_33_1_cnt,markers_34_1_cnt,markers_35_1_cnt,markers_36_1_cnt,markers_37_1_cnt,markers_38_1_cnt,markers_39_1_cnt,markers_40_1_cnt,markers_41_1_cnt,markers_42_1_cnt,markers_43_1_cnt,markers_44_1_cnt,markers_45_1_cnt,markers_47_1_cnt,markers_48_1_cnt,markers_49_1_cnt,markers_50_1_cnt,markers_51_1_cnt,markers_52_1_cnt,markers_53_1_cnt,markers_54_1_cnt,markers_55_1_cnt,markers_56_1_cnt,markers_57_1_cnt,markers_59_1_cnt,markers_60_1_cnt,markers_61_1_cnt,markers_62_1_cnt,markers_63_1_cnt,markers_64_1_cnt,markers_65_1_cnt,markers_66_1_cnt,markers_67_1_cnt,markers_68_1_cnt,markers_69_1_cnt,markers_70_1_cnt,markers_71_1_cnt,markers_72_1_cnt,markers_74_1_cnt,markers_75_1_cnt,markers_76_1_cnt,markers_77_1_cnt,markers_79_1_cnt,markers_81_1_cnt,markers_82_1_cnt,markers_83_1_cnt,markers_84_1_cnt,markers_85_1_cnt,markers_86_1_cnt,markers_87_1_cnt,markers_88_1_cnt,markers_89_1_cnt,markers_90_1_cnt,markers_91_1_cnt,markers_92_1_cnt,markers_93_1_cnt,markers_94_1_cnt,markers_95_1_cnt,markers_96_1_cnt,markers_97_1_cnt,markers_98_1_cnt,markers_99_1_cnt,markers_100_1_cnt,markers_101_1_cnt,markers_102_1_cnt,markers_103_1_cnt,markers_104_1_cnt,markers_105_1_cnt,markers_106_1_cnt,markers_107_1_cnt,markers_110_1_cnt,markers_112_1_cnt,markers_113_1_cnt,markers_114_1_cnt,markers_116_1_cnt,markers_117_1_cnt,markers_118_1_cnt,markers_119_1_cnt,markers_120_1_cnt,markers_121_1_cnt,markers_122_1_cnt,markers_124_1_cnt,markers_125_1_cnt,markers_126_1_cnt,markers_128_1_cnt,markers_129_1_cnt,markers_131_1_cnt,markers_132_1_cnt,markers_133_1_cnt,markers_135_1_cnt,markers_136_1_cnt,markers_137_1_cnt,markers_138_1_cnt,markers_139_1_cnt,markers_140_1_cnt,markers_141_1_cnt,markers_142_1_cnt,markers_143_1_cnt,markers_145_1_cnt,markers_146_1_cnt,markers_147_1_cnt,markers_148_1_cnt,markers_149_1_cnt,markers_150_1_cnt,markers_154_1_cnt,markers_155_1_cnt,markers_156_1_cnt,markers_157_1_cnt,markers_158_1_cnt,markers_159_1_cnt,markers_161_1_cnt,markers_163_1_cnt,markers_164_1_cnt,markers_165_1_cnt,markers_167_1_cnt,markers_168_1_cnt,markers_169_1_cnt,markers_170_1_cnt,markers_171_1_cnt,markers_172_1_cnt,markers_173_1_cnt,markers_174_1_cnt,markers_175_1_cnt,markers_176_1_cnt,markers_177_1_cnt,markers_178_1_cnt,markers_179_1_cnt,markers_180_1_cnt,markers_181_1_cnt,markers_182_1_cnt,markers_183_1_cnt,markers_184_1_cnt,markers_185_1_cnt,markers_187_1_cnt,markers_188_1_cnt,markers_190_1_cnt,markers_192_1_cnt,markers_193_1_cnt,markers_194_1_cnt,markers_195_1_cnt,markers_196_1_cnt,markers_198_1_cnt,markers_199_1_cnt,markers_201_1_cnt,markers_202_1_cnt,markers_203_1_cnt,markers_204_1_cnt,markers_205_1_cnt,markers_206_1_cnt,markers_207_1_cnt,markers_208_1_cnt,markers_209_1_cnt,markers_210_1_cnt,markers_211_1_cnt,markers_212_1_cnt,markers_213_1_cnt,markers_215_1_cnt,markers_217_1_cnt,markers_218_1_cnt,markers_219_1_cnt,markers_220_1_cnt,markers_221_1_cnt,markers_222_1_cnt,markers_223_1_cnt,markers_225_1_cnt,markers_226_1_cnt,markers_227_1_cnt,markers_228_1_cnt,markers_229_1_cnt,markers_230_1_cnt,markers_231_1_cnt,markers_232_1_cnt,markers_233_1_cnt,markers_234_1_cnt,markers_235_1_cnt,markers_236_1_cnt,markers_237_1_cnt,markers_238_1_cnt,markers_239_1_cnt,markers_240_1_cnt,markers_242_1_cnt,markers_243_1_cnt,markers_246_1_cnt,markers_247_1_cnt,markers_248_1_cnt,markers_249_1_cnt,markers_252_1_cnt,markers_253_1_cnt,markers_254_1_cnt,markers_255_1_cnt,markers_256_1_cnt,markers_257_1_cnt,markers_258_1_cnt,markers_260_1_cnt,markers_261_1_cnt,markers_262_1_cnt,markers_263_1_cnt,markers_264_1_

In [35]:
# удалим столбцы где много пропусков
dataset_train = dataset_train.dropna(axis=1, thresh=dataset_train.shape[0] / 2)
dataset_train = dataset_train.dropna(axis=0, thresh=dataset_train.shape[1] / 2)
dataset_train.shape

(198793, 661)

**Промежуточный итог:**

- будем обучать на channel_name == 3
- удалили константные признаки
- провели корреляционый анализ
- удалили столбцы и строки у которых NaN значений больше половины

## get_features_importances

Приступим к анализу feature_importance

In [37]:
dataset_train["target"].value_counts()

target
0    195592
1      3201
Name: count, dtype: int64

### Downsample

In [38]:
data_1 = dataset_train[dataset_train["target"] == 1]
data_0 = dataset_train[dataset_train["target"] == 0]

print(data_1.shape)
print(data_0.shape)


(3201, 657)
(195592, 657)


In [39]:
data_0_downsampled = resample(
    data_0, replace=True, n_samples=len(data_1), random_state=RANDOM_STATE
)

print(data_0_downsampled.shape)

(3201, 657)


In [40]:
data_downsampled = pd.concat([data_0_downsampled, data_1])

print(data_downsampled["target"].value_counts())


target
0    3201
1    3201
Name: count, dtype: int64


In [47]:
X_downsampled = data_downsampled.iloc[:, :-1]
y_downsampled = data_downsampled["target"]

X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(
    X_downsampled,
    y_downsampled,
    test_size=0.25,
    stratify=y_downsampled,
    random_state=RANDOM_STATE,
)

In [83]:
%%time

# модель
lgbm_model = LGBMClassifier(
    n_estimators=1000, max_depth=1, 
    learning_rate=0.01, n_jobs=-1, 
    random_state=RANDOM_STATE
)

# fit predict
lgbm_model.fit(
    X_train_d, y_train_d,
    eval_set=[(X_test_d, y_test_d)],
    eval_metric='roc_auc',
    categorical_feature=['user_lifetime_1_1_flg', 'materials_details_1_1_ctg']
)

y_pred_proba = lgbm_model.predict_proba(X_test_d)
y_pred = lgbm_model.predict(X_test_d)

# скоры
roc_score = roc_auc_score(y_test_d, y_pred_proba[:,1])
gini_score = (2 * roc_score) - 1
acc_score = accuracy_score(y_test_d, y_pred)

# выводим информацию на экран
print(f'LGBM ROC AUC SCORE: {roc_score:.2f}')
print(f'LGBM GINI SCORE: {gini_score:.2f}')
print(f'LGBM ACCURACY SCORE: {acc_score:.2f}')
print('-' * 25)

C:\Users\Nikolaj\anaconda3\envs\hackaton-r1\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\Nikolaj\anaconda3\envs\hackaton-r1\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['materials_details_1_1_ctg', 'user_lifetime_1_1_flg']
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\Nikolaj\anaconda3\envs\hackaton-r1\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\Nikolaj\anaconda3\envs\hackaton-r1\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[1]	valid_0's binary_logloss: 0.692905
[2]	valid_0's binary_logloss: 0.692668
[3]	valid_0's binary_logloss: 0.692435
[4]	valid_0's binary_logloss: 0.692298
[5]	valid_0's binary_logloss: 0.692071
[6]	valid_0's binary_logloss: 0.691938
[7]	valid_0's binary_logloss: 0.691717
[8]	valid_0's binary_logloss: 0.691589
[9]	valid_0's binary_logloss: 0.691373
[10]	valid_0's binary_logloss: 0.69125
[11]	valid_0's binary_logloss: 0.691038
[12]	valid_0's binary_logloss: 0.690919
[13]	valid_0's binary_logloss: 0.690714
[14]	valid_0's binary_logloss: 0.690599
[15]	valid_0's binary_logloss: 0.690398
[16]	valid_0's binary_logloss: 0.690256
[17]	valid_0's binary_logloss: 0.690137
[18]	valid_0's binary_logloss: 0.690028
[19]	valid_0's binary_logloss: 0.689836
[20]	valid_0's binary_logloss: 0.6897
[21]	valid_0's binary_logloss: 0.689588
[22]	valid_0's binary_logloss: 0.6894
[23]	valid_0's binary_logloss: 0.68927
[24]	valid_0's binary_logloss: 0.68917
[25]	valid_0's binary_logloss: 0.688989
[26]	valid_0's b

[582]	valid_0's binary_logloss: 0.669123
[583]	valid_0's binary_logloss: 0.66911
[584]	valid_0's binary_logloss: 0.669097
[585]	valid_0's binary_logloss: 0.669069
[586]	valid_0's binary_logloss: 0.669058
[587]	valid_0's binary_logloss: 0.669053
[588]	valid_0's binary_logloss: 0.669059
[589]	valid_0's binary_logloss: 0.669048
[590]	valid_0's binary_logloss: 0.669018
[591]	valid_0's binary_logloss: 0.668997
[592]	valid_0's binary_logloss: 0.668982
[593]	valid_0's binary_logloss: 0.668988
[594]	valid_0's binary_logloss: 0.669002
[595]	valid_0's binary_logloss: 0.668996
[596]	valid_0's binary_logloss: 0.668977
[597]	valid_0's binary_logloss: 0.668949
[598]	valid_0's binary_logloss: 0.66894
[599]	valid_0's binary_logloss: 0.668927
[600]	valid_0's binary_logloss: 0.66893
[601]	valid_0's binary_logloss: 0.668936
[602]	valid_0's binary_logloss: 0.668921
[603]	valid_0's binary_logloss: 0.668885
[604]	valid_0's binary_logloss: 0.668875
[605]	valid_0's binary_logloss: 0.668862
[606]	valid_0's bin

LGBM ROC AUC SCORE: 0.63
LGBM GINI SCORE: 0.27
LGBM ACCURACY SCORE: 0.60
-------------------------
CPU times: total: 6.84 s
Wall time: 662 ms


In [54]:
%%time

# задаем сетку параметров
param_grid_lgbm = {
    'n_estimators': range(100, 1001, 100),
    'max_depth': range(1, 5, 1),
    'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2]
}

# описываем gs
gs_lgbm_pl = GridSearchCV(
    lgbm_model, 
    param_grid=param_grid_lgbm, 
    scoring='roc_auc', 
    n_jobs=-1,
    cv=5
)

# фиттим модель
gs_lgbm_pl.fit(X_train_d, y_train_d)

# сохраняем лучший скор и параметры
gs_lgbm_best_score = gs_lgbm_pl.best_score_
gs_lgbm_best_params = gs_lgbm_pl.best_params_


# выводим информацию на экран
print(f'LGBM BEST ROC AUC SCORE: {gs_lgbm_best_score:.2f}')
print(f'LGBM BEST PARAMS: {gs_lgbm_best_params}')
print('-' * 93)

LGBM BEST ROC AUC SCORE: 0.64
LGBM BEST PARAMS: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 1000}
---------------------------------------------------------------------------------------------
CPU times: total: 11.4 s
Wall time: 5min 56s


In [45]:
# Получите важности признаков
feature_importance = lgbm_model.feature_importances_

# Создайте DataFrame с названиями признаков и их важностями
importance_df = pd.DataFrame(
    {"feature": X_train.columns, "importance": feature_importance}
)

# Отсортируйте по убыванию важности
importance_df = importance_df.sort_values("importance", ascending=False)

# Выведите результаты
importance_df.head(15)

,feature,importance
653,payments_details_49_6_avg,66
648,charges_details_17_6_avg,61
647,charges_details_12_1_sum,60
641,payments_details_23_3d6_avg,57
649,payments_details_34_3_sum,56
640,payments_details_19_1d6_avg,52
646,balance_details_0_1_num,48
652,payments_details_47_3_avg,47
245,markers_334_1_cnt,47
87,markers_104_1_cnt,47


### class_weight = 'balanced'

In [50]:
X = dataset_train.iloc[:, :-1]
y = dataset_train["target"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, stratify=y, random_state=RANDOM_STATE
)

In [85]:
%%time

# модель
lgbm_model = LGBMClassifier(n_jobs=-1, class_weight='balanced', random_state=RANDOM_STATE)

# fit predict
lgbm_model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    eval_metric='roc_auc',
    categorical_feature=['user_lifetime_1_1_flg', 'materials_details_1_1_ctg']
)

y_pred_proba = lgbm_model.predict_proba(X_test)
y_pred = lgbm_model.predict(X_test)

# скоры
roc_score = roc_auc_score(y_test, y_pred_proba[:,1])
gini_score = (2 * roc_score) - 1
acc_score = accuracy_score(y_test, y_pred)

# выводим информацию на экран
print(f'LGBM ROC AUC SCORE: {roc_score:.2f}')
print(f'LGBM GINI SCORE: {gini_score:.2f}')
print(f'LGBM ACCURACY SCORE: {acc_score:.2f}')
print('-' * 25)

C:\Users\Nikolaj\anaconda3\envs\hackaton-r1\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\Nikolaj\anaconda3\envs\hackaton-r1\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['materials_details_1_1_ctg', 'user_lifetime_1_1_flg']
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\Nikolaj\anaconda3\envs\hackaton-r1\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\Nikolaj\anaconda3\envs\hackaton-r1\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[1]	valid_0's binary_logloss: 0.68513
[2]	valid_0's binary_logloss: 0.677898
[3]	valid_0's binary_logloss: 0.671548
[4]	valid_0's binary_logloss: 0.666522
[5]	valid_0's binary_logloss: 0.66153
[6]	valid_0's binary_logloss: 0.657281
[7]	valid_0's binary_logloss: 0.652523
[8]	valid_0's binary_logloss: 0.648247
[9]	valid_0's binary_logloss: 0.644603
[10]	valid_0's binary_logloss: 0.641207
[11]	valid_0's binary_logloss: 0.637929
[12]	valid_0's binary_logloss: 0.634227
[13]	valid_0's binary_logloss: 0.631345
[14]	valid_0's binary_logloss: 0.628021
[15]	valid_0's binary_logloss: 0.624598
[16]	valid_0's binary_logloss: 0.621369
[17]	valid_0's binary_logloss: 0.618376
[18]	valid_0's binary_logloss: 0.615611
[19]	valid_0's binary_logloss: 0.612712
[20]	valid_0's binary_logloss: 0.609906
[21]	valid_0's binary_logloss: 0.607548
[22]	valid_0's binary_logloss: 0.604845
[23]	valid_0's binary_logloss: 0.602147
[24]	valid_0's binary_logloss: 0.599557
[25]	valid_0's binary_logloss: 0.596947
[26]	valid_

In [87]:
# Получите важности признаков
feature_importance = lgbm_model.feature_importances_

# Создайте DataFrame с названиями признаков и их важностями
importance_df = pd.DataFrame(
    {"feature": X_train.columns, "importance": feature_importance}
)

# Отсортируйте по убыванию важности
importance_df = importance_df.sort_values("importance", ascending=False)

# Выведите результаты
importance_df.head(30)

,feature,importance
648,charges_details_17_6_avg,59
653,payments_details_49_6_avg,58
646,balance_details_0_1_num,47
641,payments_details_23_3d6_avg,47
649,payments_details_34_3_sum,44
640,payments_details_19_1d6_avg,43
651,payments_details_45_1_avg,38
256,markers_346_1_cnt,37
652,payments_details_47_3_avg,36
647,charges_details_12_1_sum,34


### Отбираем признаки

In [88]:
for_now = importance_df["feature"].head(30).to_list()
for_now

['charges_details_17_6_avg',
 'payments_details_49_6_avg',
 'balance_details_0_1_num',
 'payments_details_23_3d6_avg',
 'payments_details_34_3_sum',
 'payments_details_19_1d6_avg',
 'payments_details_45_1_avg',
 'markers_346_1_cnt',
 'payments_details_47_3_avg',
 'charges_details_12_1_sum',
 'markers_104_1_cnt',
 'markers_330_1_cnt',
 'markers_324_1_cnt',
 'markers_4_1_cnt',
 'markers_508_1_cnt',
 'markers_348_1_cnt',
 'markers_706_1_cnt',
 'markers_349_1_cnt',
 'markers_772_1_cnt',
 'markers_542_1_cnt',
 'markers_238_1_cnt',
 'markers_242_1_cnt',
 'markers_334_1_cnt',
 'markers_103_1_cnt',
 'markers_72_1_cnt',
 'markers_306_1_cnt',
 'markers_506_1_cnt',
 'materials_details_1_1_ctg',
 'markers_535_1_cnt',
 'markers_146_1_cnt']

In [ ]:
# сохраняем результаты
top_50_numerical = importance_df["feature"].head(50).to_list()

In [89]:
# Получение матрицы ошибок
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

# Создание pd.DataFrame для матрицы ошибок
data = {"Прогноз (+)": [tp, fp], "Прогноз (-)": [fn, tn]}
index = ["Реальность (+)", "Реальность (-)"]
conf_matrix = pd.DataFrame(data=data, index=index)

# Вывод матрицы ошибок
print(conf_matrix)

                Прогноз (+)  Прогноз (-)
Реальность (+)          286          514
Реальность (-)         9844        39055
